# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [79]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Read csv, store it as dataframe, and display df head
cities_df = pd.read_csv("Resources/city_weather_data.csv")
cities_df.head()

,Unnamed: 0,City,Latitude,Longitude,Max. Temp (F),Humidity (%),Cloudiness (%),Wind Speed (MPH),Country,Date
0,0,rikitea,-23.1203,-134.9692,73.54,74,36,17.83,PF,1627323638
1,1,norman wells,65.2820,-126.8329,60.84,63,75,11.50,CA,1627323638
2,2,atuona,-9.8000,-139.0333,77.52,70,55,25.08,PF,1627323638
3,3,pitkyaranta,61.5734,31.4792,63.52,81,55,3.31,RU,1627323639
4,4,mataura,-46.1927,168.8643,42.48,98,91,2.95,NZ,1627323114


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
# Create heatmap
fig = gmaps.figure()

# Assign values for location and humidity, which will be used at location and
# weight
locations = cities_df[["Latitude", "Longitude"]]
humidity = cities_df["Humidity (%)"]

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

# Add heat layer
fig.add_layer(heat_layer)

# Display heatmap
fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [63]:
# I will be using the following weather criteria:
# Temperature below 80 F
# Humidity below 70%
# Wind speed below 10 mph

# Create filtered df using loc
filtered_cities_df = cities_df.loc[(cities_df["Max. Temp (F)"] <= 80) &
                                  (cities_df["Humidity (%)"] <= 70) &
                                  (cities_df["Wind Speed (MPH)"] <= 10)]

filtered_cities_df = filtered_cities_df.reset_index()

# Display df
filtered_cities_df

,index,Unnamed: 0,City,Latitude,Longitude,Max. Temp (F),Humidity (%),Cloudiness (%),Wind Speed (MPH),Country,Date
0,5,5,upernavik,72.7868,-56.1549,45.81,63.0,100,4.59,GL,1627323639
1,7,7,pisco,-13.7000,-76.2167,70.02,68.0,0,6.91,PE,1627323640
2,19,19,vila franca do campo,37.7167,-25.4333,72.01,66.0,0,6.22,PT,1627323643
3,37,37,lakes entrance,-37.8811,147.9810,57.18,57.0,100,9.24,AU,1627323648
4,39,39,fort nelson,58.8053,-122.7002,69.39,52.0,20,5.75,CA,1627323649
...,...,...,...,...,...,...,...,...,...,...,...
93,523,523,diego de almagro,-26.3667,-70.0500,67.08,17.0,0,7.25,CL,1627323802
94,538,538,cotacachi,0.3000,-78.2667,67.55,57.0,85,5.03,EC,1627323807
95,550,550,karaman,37.0833,33.2500,72.61,31.0,17,8.99,TR,1627323812
96,553,553,katakwi,1.8911,33.9661,71.15,58.0,100,3.51,UG,1627323813


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [131]:
# Setup hotel_df, with columns for hotel name, lat, lng, and city
# Initialize 'city' column with cities from filtered_cities_df
hotel_df = pd.DataFrame(columns=["Hotel Name", "Latitude", "Longitude", "City", "Country"])
hotel_df["City"] = filtered_cities_df["City"]
hotel_df["Country"] = filtered_cities_df["Country"]
hotel_df = hotel_df.reset_index()

# Setup for query
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": "",
    "rankyby": "distance",
    "type": "hotel",
    "keyword": "hotel",
    "radius": 5000,
    "key": g_key
}


# Loops through hotel_df and finds closest hotel to city's listed coordinates
for index, row in hotel_df.iterrows():
    # Gets city coordinates from filtered_city_df
    lat = float(filtered_cities_df.loc[filtered_cities_df["City"] == row.City,"Latitude"])
    lng = float(filtered_cities_df.loc[filtered_cities_df["City"] == row.City, "Longitude"])

    # Sets city coordinates in paramters
    params["location"] = f"{lat}, {lng}"

    # Gets query and converts it to json
    get_hotel = requests.get(base_url, params=params).json()
    
    # Finds latitude, longitude, and hotel name and appends it to dataframe
    try:
        print(f"Finding closest hotel in {row.City}")
        
        hotel_df.loc[index, "Latitude"] = get_hotel["results"][1]["geometry"]["location"]["lat"]
        hotel_df.loc[index, "Longitude"] = get_hotel["results"][1]["geometry"]["location"]["lng"]
        hotel_df.loc[index, "Hotel Name"] = get_hotel["results"][1]["name"]
        
        
    # If this information is not found, or there is an error, returns the following
    except:
        print("Cannot find hotel in this city.")

# Display df
hotel_df

Finding closest hotel in upernavik
Finding closest hotel in pisco
Finding closest hotel in vila franca do campo
Finding closest hotel in lakes entrance
Finding closest hotel in fort nelson
Finding closest hotel in narsaq
Finding closest hotel in san cristobal
Finding closest hotel in hermanus
Finding closest hotel in mount isa
Finding closest hotel in yangambi
Cannot find hotel in this city.
Finding closest hotel in mosquera
Finding closest hotel in ulladulla
Finding closest hotel in bredasdorp
Finding closest hotel in airai
Cannot find hotel in this city.
Finding closest hotel in thompson
Finding closest hotel in suntar
Finding closest hotel in teya
Cannot find hotel in this city.
Finding closest hotel in luderitz
Finding closest hotel in henties bay
Finding closest hotel in esil
Finding closest hotel in kipushi
Cannot find hotel in this city.
Finding closest hotel in masvingo
Finding closest hotel in nikolskoye
Cannot find hotel in this city.
Finding closest hotel in ambilobe
Cannot 

,index,Hotel Name,Latitude,Longitude,City,Country
0,0,Café de Upernavik,72.781921,-56.151283,upernavik,GL
1,1,HOSTAL MIRAMAR,-13.727375,-76.222341,pisco,PE
2,2,Vinha d'Areia Beach Hotel,37.716274,-25.42878,vila franca do campo,PT
3,3,Abel Tasman Waterfront Motel Lakes Entrance,-37.877258,147.999459,lakes entrance,AU
4,4,Woodlands Inn & Suites,58.800781,-122.676784,fort nelson,CA
...,...,...,...,...,...,...
93,93,Hotel Sol Del Desierto,-26.388343,-70.047184,diego de almagro,CL
94,94,Land Of The Sun Hotel & Spa,0.300568,-78.265357,cotacachi,EC
95,95,NaN,NaN,NaN,karaman,TR
96,96,Eden Guest House,1.918181,33.952975,katakwi,UG


In [134]:
# Cleanup of df - drop rows where a hotel cannot be found and resets index
hotel_df = hotel_df.dropna()
hotel_df = hotel_df.reset_index()

# Display df
hotel_df

,level_0,index,Hotel Name,Latitude,Longitude,City,Country
0,0,0,Café de Upernavik,72.781921,-56.151283,upernavik,GL
1,1,1,HOSTAL MIRAMAR,-13.727375,-76.222341,pisco,PE
2,2,2,Vinha d'Areia Beach Hotel,37.716274,-25.42878,vila franca do campo,PT
3,3,3,Abel Tasman Waterfront Motel Lakes Entrance,-37.877258,147.999459,lakes entrance,AU
4,4,4,Woodlands Inn & Suites,58.800781,-122.676784,fort nelson,CA
...,...,...,...,...,...,...,...
63,92,92,ibis Weinan Normal University Chaoyang Street ...,34.497204,109.4933,weinan,CN
64,93,93,Hotel Sol Del Desierto,-26.388343,-70.047184,diego de almagro,CL
65,94,94,Land Of The Sun Hotel & Spa,0.300568,-78.265357,cotacachi,EC
66,96,96,Eden Guest House,1.918181,33.952975,katakwi,UG


In [135]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_df = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: "None of [Index(['Lat', 'Lng'], dtype='object')] are in the [columns]"

In [ ]:
# Add marker layer ontop of heat map


# Display figure
